### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Assigning data to variables
total_schools = len(school_data_complete["school_name"].unique())
total_students = (school_data_complete["student_name"].count())
total_budget = school_data["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()
pass_math = 100*school_data_complete["math_score"].loc[(school_data_complete["math_score"]>=70)].count()/total_students
pass_read = 100*school_data_complete["reading_score"].loc[(school_data_complete["reading_score"]>=70)].count()/total_students
pass_overall = 100*school_data_complete["math_score"].loc[(school_data_complete["math_score"]>=70)&(school_data_complete["reading_score"]>=70)].count()/total_students

#Store in dataframe and format
district_summary_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":["{:,}".format(total_students)],
    "Total Budget":["${:,.2f}".format(total_budget)],
    "Average Math Score":[avg_math_score],
    "Average Reading Score":[round(avg_read_score,5)],
    "% Passing Math":[pass_math],
    "% Passing Reading":[pass_read],
    "% Overall Passing":[pass_overall]
})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#Add Passing columns (1 = pass, 0 = fail) for combined DF
school_data_complete["pass math"] = np.where(school_data_complete["math_score"]>=70,1,0)
school_data_complete["pass reading"] = np.where(school_data_complete["reading_score"]>=70,1,0)
school_data_complete["pass overall"] = np.where((school_data_complete["math_score"]>=70)&(school_data_complete["reading_score"]>=70),1,0)

#Get scores averages and pass % per school
scores_by_school = school_data_complete.groupby("school_name")[["reading_score","math_score","pass math","pass reading","pass overall"]].agg('mean')

#Merge with school data
school_summary = pd.merge(school_data, scores_by_school, how="left", on=["school_name", "school_name"])

#Add per student budget
school_summary["per_student_budget"] = school_summary["budget"]/school_summary["size"]

#Multiply pass by 100
school_summary["pass math"] = school_summary["pass math"]*100
school_summary["pass reading"] = school_summary["pass reading"]*100
school_summary["pass overall"] = school_summary["pass overall"]*100

#Filter columns
school_summary_clean = school_summary[["school_name","type","size","budget","per_student_budget","math_score","reading_score","pass math","pass reading","pass overall"]]

#Rename columns
school_summary_renamed = school_summary_clean.rename(columns={
    "school_name":"",
    "type":"School Type",
    "size":"Total Students",
    "budget":"Total School Budget",
    "per_student_budget":"Per Student Budget",
    "math_score":"Average Math Score",
    "reading_score":"Average Reading Score",
    "pass math":"% Passing Math",
    "pass reading":"% Passing Reading",
    "pass overall":"% Overall Passing"
})

#Format and print
school_summary_format = school_summary_renamed.style.format({
    'Total School Budget':'${:,.2f}',
    'Per Student Budget':'${:,.2f}',
})
school_summary_format.hide_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [11]:
#Sorting school_summary_renamed by Overall Passing
top_overall = school_summary_renamed.sort_values("% Overall Passing", ascending=False)

#Format and print
top_overall_format = top_overall.head().style.format({
    'Total School Budget':'${:,.2f}',
    'Per Student Budget':'${:,.2f}',
})
top_overall_format.hide_index()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type